# Arithmetic coding applied to transform coefficients
In the last module we've seen how to make Elias coding practical and build an arithmetic codec which:
 * Uses integer arithmetic only.
 * Does not require multiplications.

Such a codec could be used as the ***core engine*** of the entropy coding stage. The question is now: *how can we use such a core engine in a practical image codec*? We will consider in the following an image compression algorithm like the one specified by the Joint Photographic Experts Group (JPEG) compression standard in its *baseline* - sequential processing (\*). Replicating the full algorithm is out of the scope of this tutorial; we'll instead focus our consideration around the entropy coding of data associated with an 8$\times$8 Discrete Cosine Transform (DCT) block, passed through a uniform quantiser as specified in the JPEG compression standard.

(\*) To be really precise and avoid creating confusion, we shall clarify that the baseline processing of the JPEG compression standard uses the well-known run length Huffman encoding with alphabet extension. The use of arithmetic coding in JPEG is specified in [Annex D](https://www.itu.int/rec/T-REC-T.81) of the standard document and, to the best of my knowledge, it has been rarely used in practical applications. Nevertheless the binary arithmetic coding engine is a good benchmark to study the subject and, it is also worth mentioning, that it has been adopted in the newest MPEG-5 Essential Video Coding (EVC) standard.

Roughly speaking, two main aspects need to be addressed here:
 * How the probability of the Least (Less) Probable Symbol (LPS) can be estimated and updated.
 * How we take advantage from the fact that we're encoding data associated with quantised transform coefficients.

We'll address these two aspects in the following, starting with the estimate and update of the LPS probability.

## Estimating the LPS probability
The probability for the LPS needs to be estimated and updated so that the arithmetic coding engine can adapt to the source's statistics. If we don't know anything about the source being encoded, we could set such a probability, say $p_{LPS}$, to 0.5 and then keep two counters to accumulate the number of times the input is an LPS or MPS, $n_{LPS}$, $n_{MPS}$. Then the $p_{LPS}$ is given by:

$$
\large
p_{LPS} = \frac{n_{LPS}}{n_{LPS} + n_{MPS}},
$$

and consequently $p_{MPS} = 1 - p_{LPS}$.

However this approach suffers by two fundamental drawbacks:
 * It is slow in adapting the coding engine to the signal statistics, and more importantly.
 * It is still considering that our source is i.i.d (independent and identically distributed).
 
Quantised transform coefficients associated with images are not i.i.d and their *inter-symbol* dependency is also found at bit plane granularity level. Accordingly, the probability that the current bit is going to be an LPS strongly depends on the sequence of bits observed so far.

We can define a set of states, each representing a given sequence of bits and associate each state (say, $\sigma$) with a given probability $p_{LPS}(\sigma)$.

Different states $\sigma_1,\sigma_2,\ldots,\sigma_N$ are connected with branches associated with the read from the input of an LPS or MPS. The different transitions among states can be defined by a probability law whose parameters are fitted on real data. An example of such state transition graph is reported below for the [CABAC](http://iphome.hhi.de/wiegand/assets/pdfs/csvt_cabac_0305.pdf) (Content Adaptive Binary Arithmetic Coding) of the H.264/AVC standard.

<img src="cabac-state-machine.png" width="900">

The graph tells us that if the encoding engine continues to the be provided with MPS symbols (solid black edges) then $p_{LPS}$ decreases since we're building up a long sequence of MPS bits, hence we're heading towards the right hand side of the transition graph. Red dotted lines are the equivalent of *goto* jumps in programming languages which would increase our $p_{LPS}$.

With the probability transition graph above we can estimate the probability of the current bit being an LPS or MPS given the previously observed bits emitted. We also note that such a graph has a general validity. That is it can be used for a large variety of patterns of bits (from near to random sequences of LPS and MPS, to stable LPS / MPS strings).

We also note that the types of bit patterns depend on the source being encoded (e.g. quantised coefficients of luma and chroma samples). Accordingly, it is better to keep track of the *state evolution* in the graph for each different type of source we're encoding. Such tracking is realised using a variable called **context**.

The picture below shows an example on how a context can be organised in one possible implementation of our arithmetic coding engine. We have:
 * Probability of the LPS
 * The index of the state where the data distribution sits in the transition graph
 * The semantics associated with the LPS (that is zero or one)

<img src="context-structure.png" width="400">

We can have as many contexts as needed to tackle data source which are spatio and/or temporal variant like images and videos. The trade-off is between memory requirements and coding efficiency.

Also, very importantly, care should be taken to avoid the so-called **context dilution** effect where we allocate contexts for data samples which are associated with noise: in that case we would have very few samples to obtain a good estimate of the probability.

Each context is associated with a given binary source. Since we encode quantised coefficients which are not binary quantities we need to *binarise* them and then apply binary arithmetic coding. There are plenty of ways of doing this, the next paragraph reports the binarisation specified by the JPEG compression standard.

## Entropy coding of DCT coefficients with binary arithmetic coding
Two main types of coefficients need entropy coding:
 * DC coefficients (luma and chroma).
 * AC coefficients (luma and chroma).
 
 The JPEG compression standard allows to specify and use a different set of contexts for each colour component. Whether to use a single set of contexts for all colour components or a dedicated set for each one, is indicated in the *Define Arithmetic Coding* (DAC) marker segment, described in Section [B.2.4.3](https://www.itu.int/rec/T-REC-T.81) of the JPEG compression format. We'll review in the following the binarisation and use of contexts for the DC and AC coefficients, starting with the DC ones first.

### Binarisation and context allocation for DC coefficients
The JPEG compression standard applies Differential Pulse Code Modulation (DPCM) over the quantised DC levels. This may lead to a prediction error which either:
 * Oscillates around zero.
 * Slowly deviates from zero (with positive or negative values).
 * Largely deviates from zero (with positive or negative values)..
 
Depending on the statistics of the content being compressed we might have a distribution of the differential DC ($\Delta$DC) which is a mix of the three flavours above. The picture below shows an example of such a combined distribution

<img src="multi-variate-distribution.png" width="1000">

Moreover, we note from the picture above that locally the distribution of data can be different (e.g. Laplacian, uniform or something in between). Using the **same** probability model (i.e. context) over different ranges of data is **not a good idea**.

Accordingly, we could define a context associated with each of the different situations depicted above to better adpat the arithmetic encoding process. This poses a *chicken and egg dilemma*: to select the right context the decoder needs to know the value of the $\Delta$DC coefficient but it hasn't decoded it yet (because it doesn't know which context to select).

One way to solve this is to consider the previously decoded $\Delta$DC value (i.e. the one from the previous 8$\times$8 DCT block). Generally DC values (and associated $\Delta$DC) are correlated across spatially adjacent blocks, thus using the previously decoded $\Delta$DC would provide a reasonable estimate of the value for the current difference as well as the probability model (i.e. the context) to use for arithmetic coding. 

The JPEG compression standard allows the user to specify the bounds whereas a value is classified as *zero*, *small* or *large*. These parameters are denoted as $L$ and $U$ and can be transmitted in the aforementioned DAC marker segment with the following meaning:

$$
\large
small(v) \equiv \left\{
\begin{array}{ll}
v \in (0, 2^U] & L = 0\\
v \in (2^{L-1}, 2^U) & L > 0
\end{array}\right.
$$

The current $\Delta$DC value can then be: zero, positive, negative, greater than 2 or less than -2. The decision on whether $\Delta$DC belongs to one of these value ranges is a binary one and the JPEG compression standard specifies four different contexts as follows:
 * 1 context to encode whether $\Delta$DC is different from zero
 * 1 context to encode the sign of $\Delta$DC
 * 1 context to encode whether $\Delta$DC is greater than 2
 * 1 context to encode whether $\Delta$DC is less than -2
 
Worth mentioning here that the last two contexts above are actually associated with the binary decision $\Delta$DC > $\pm1$, since for a non zero coefficient, the JPEG compression standard prescribes to transform its value $V$ to $V \leftarrow |V| - 1$ (cmp. Figure F.6 of the [specification document](https://www.itu.int/rec/T-REC-T.81)). The four contexts above are used for a specific range of values of the previous $\Delta$DC, thus there will be a total of 4 $\times$ 5 = 20 contexts as depicted in the following figure.

<img src="dc-context-first-part.png" width="1000">

When the value $V$ is greater than 1, the JPEG compression standard performs arithmetic encoding by encoding the bitplane value of its Most Significant Bit (MSB) and the binary representation of the remaining less significant bits. The bitplane value associated with the MSB is a number in the range \[0, 15) which is binarised using a sequence of zeros or ones, usually denoted as *bins*, where bin $i$ answers the binary question: *is the magnitude of the current value less than* $2^i$? A *no* answer to the question is codified with a zero and the binarisation process terminates whenever the answer to the question is *yes* (codified with a 1).

Consider for example $V = 9$, the bitplane value of its MSB is 4, its binarisation would generate the string `0001`. Four contexts are used here and we note that the more values will have their MSB bitplane equal to four, the more the distribution of the LPS in each context will become skewed, making the arithmetic entropy coding engine more efficient, i.e. encoding symbols with a fractional number of bits. As mentioned above, the remaining less significant bits of the binary code for 9 (3 in this case, `001`) can be encoded just using their value and having a single context since they're likely to be uniformly distributed. It makes instead sense to separate such a single context with respect to the value of MSB's bitplane, since the extent of the uniform distribution around each bitplane may be different.

Imagining ourselves writing the sequence of calls to the entropy coding engine we would have something like the following (where a Python pseudo-code syntax is assumed):
```python
code0(c1)  # MSB of 9 is less than 2^1?
code0(c2)  # MSB of 9 is less than 2^2?
code0(c3)  # MSB of 9 is less than 2^3?
code1(c4)  # MSB of 9 is less than 2^4?
code0(c_4lsb) # Binary string for 001
code0(c_4lsb) # Binary string for 001
code1(c_4lsb) # Binary string for 001
```

Where `ci` denotes the context associated with the $i$-th bitplane whilst `c_4lsb` is the context associated with the less significant bits around bitplane equal to 4: we use one context since, as mentioned above, the distribution is going to be fairly uniform. You might be tempted to state that a four bit quantity (i.e. $V = 9$) is going to encoded with 7 bits (i.e. `0001001`), hence we'd have been better off just writing the four bits associated with its binary encoding. This is not the case since the coding of each bin will require less than 1 bit when the contexts will be settled with the right probabilities.

To perform binary encoding of the DC coefficients in the JPEG compression standard we would need the following grand total of coefficients:
 * 20 contexts as previously mentioned to account for $\Delta$DC being zero or not, conditioned to the value of the previously encoded $\Delta$DC.
 * 15 contexts associated with the bitplane value of the MSB.
 * 14 contexts for the less significant bits associated with each value of the MSB's bitplane.
 * Total = 49.
 
In case the set of contexts is used on a per colour component basis, the amount of memory required can quickly add up. Generally speaking a good practice would corresponds to use a set of context for the luma and one for the two chrominance components. The following figure shows a possible organisation of the 49 contexts in a 1D array.

<img src="dc-context-second-part.png" width="1000">

### Binarisation and context allocation for AC coefficients
AC coefficients in an 8$\times$8 DCT block are scanned according to a zig-zag pattern which allows to rank the transform coefficients in a descending order of their absolute value. When an AC coefficient is different from zero and greater than or less than 1 or -1, its value is encoded following the same process explained above for $\Delta$DC, that is:
 * Arithmetic coding of the binary flag associated with the coefficient's sign.
 * Binarisation of the bitplane value associated with the MSB and arithmetic encoding of each string bin.
 * Arithmetic encoding of the less significant bits.

If a coefficient instead is zero, then only a single call to the entropy coding engine is performed to encode the binary flag. Moreover, if a coefficient is not zero, the JPEG compression standard requires to encode a binary flag which denotes whether it is the End Of Block (EOB) coefficient or not, that is the last non zero coefficient in the 8$\times$8 DCT block.

Depending on the image statistics and coding rate, the value of the following three quantities can be strongly dependent on the coefficient position in the 8$\times$8 DCT block:
 * The coefficient being the EOB.
 * The coefficient being zero.
 * The coefficient being greater than or less than one.
 
Accordingly, the JPEG compression format specifies for each of the 63 AC coefficients, three different contexts for each of these quantities. We will have an initial total of contexts equal to 189 (i.e. 63 $\times$ 3).

On top of this initial total we need to add the 14 context associated with the bitplane value of the MSB and then the 14 contexts associated with the binary encoding of the less significant bits. This would then lead to a total of 217 contexts. The JPEG standard allows an additional level of adaptability of the entropy coding process by conditioning the set of contexts used for the bit plane and less significant bits on the position of the AC coefficient in the 8$\times$8 DCT block. In particular, the standard specifies a threshold ($Kx$, see Section [F.1.4.4.2](https://www.itu.int/rec/T-REC-T.81) of the specification document for more details) on the coefficient position $k$ whereby if $k \leq Kx$, then a first of set of 28 contexts is used and a second one otherwise. The value of $Kx$ is transmitted in the DAC segment marker and is assumed to be 5 in case is no DAC marker is present in the bitstream. When these two sets of 28 contexts each are considered the grand total of contexts used to entropy encoder the AC coefficient amounts to 245 contexts. This base number can scale up if each colour component is encoded with a different set of contexts.

## Play time
We'll now apply arithmetic coding to a set of data distributed according to a zero-mean [Laplacian](https://en.wikipedia.org/wiki/Laplace_distribution) probability mass function. As we already know this kind of distribution is well suited to model data associated with quantised transform coefficients.

The Python code cell below shows an example of generation of such data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

mu, l = 0, 10

x = np.random.uniform(low=-0.5, high=0.5, size=1000)

lx = mu - l * np.sign(x) * np.log(1 - 2 * np.abs(x))
lx = np.round(lx)
plt.hist(lx, bins=30)
plt.grid()
plt.xlabel('$x$', fontsize=16)
plt.ylabel(r"$\frac{\lambda}{2}\cdot e^{(-\lambda\cdot|x - \mu|)}$", fontsize=16);

For the sake of results reproducibility, the repository contains a pre-generated set of samples where arithmetic coding will be applied.

We'll use the QM coder from the JPEG compression standard, given that we focused the whole tutorial on this format.

The Python code cell below loads the package implementing the QM codec along with the data needed for this example. We assume integer data in the range \[-8, 7\] distributed according to a zero mean Laplacian pmf, representing quantised transform coefficients.

In [ ]:
data = np.loadtxt('laplacian-data.csv', delimiter=',', dtype=int)
original_bps = 4
total_data = len(data)

plt.hist(data, bins=[-8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7])
plt.grid()
plt.xlabel('$x$', fontsize=16)
plt.ylabel(r"PMF($x$)", fontsize=16);

We're using a binary arithmetic coder, thus we'd need to perform a binarisation of our data. Before we crack on such a process, we make few observations:
 * We know that the most frequent value is equal to zero (it's a zero-mean Laplacian distribution).
 * Because of the symmetry of the probability mass function, the sign associated with each symbol is likely to have equal chance to be positive or negative, i.e. is likely to follow a uniform distribution.
 * Yet because is a Laplace distribution, we notice that the values decrease (rapidly) as we move away from zero (in both directions on the horizontal axis).
The following Python code cell, implements a first example of binarisation and arithmetic coding applied to our data.

In [ ]:
from qmencoder import QMEncoder
from qmutilities import Context, endOfArithmeticCodingMarker

qme = QMEncoder()
qme.init_enc()

significance_ctx = Context(0, 0)
sign_ctx = Context(113, 0)
lsb_ctx = Context(113, 0)
for d in data:
    if not d:
        qme.code0(significance_ctx)
    else:
        qme.code1(significance_ctx)
        # Code the sign
        qme.code0(sign_ctx) if d > 0 else qme.code1(sign_ctx)
        d = abs(d) - 1
        bins = (bin(d)[2:]).zfill(3)
        for b in bins:
            qme.code0(lsb_ctx) if b == '0' else qme.code1(lsb_ctx)

qme.flush()
qme.insert_marker(endOfArithmeticCodingMarker)

byte_buffer = qme.get_byte_buffer()

original_bits = total_data * original_bps
coding_rate = len(byte_buffer) * 8
bps = coding_rate / total_data
print(f"Compression ratio: {original_bits / coding_rate:.2f}, bits per symbol: {bps}")

By comparing this result with the Shannon's entropy:

In [ ]:
p, _ = np.histogram(data, bins=[-8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7], density=True)
H = -np.sum(p * np.log2(p))
print(f"Source entropy: {H:.2f} bits per symbol")

We see that we're not quite there with the performance of our arithmetic coding scheme. So the challenge for you is to accomplish the following:
 * Look at the code above and find a design flaw (it's not a bug per se but rather an odd choice).
 * Extend the code to take even more advantage of the data distribution.

For the sake of completeness we also report the decoding algorithm, provided that it needs to be modified in light of the changes made to improve the coding efficiency.

In [ ]:
from qmdecoder import QMDecoder
from numpy.testing import assert_array_equal

print(f"Starting the decoding process...")

qmd = QMDecoder()
qmd.initDec(byte_buffer)

significance_ctx = Context(0, 0)
sign_ctx = Context(113, 0)
lsb_ctx = Context(113, 0)
decoded_data = np.zeros((total_data), np.int32)

for i in range(total_data):
    significant = qmd.decode(significance_ctx, byte_buffer)
    if significant:
        current_sign = 1 if qmd.decode(sign_ctx, byte_buffer) == 0 else -1
        decoded_lsb = 0
        for _ in range(3):
            b = qmd.decode(lsb_ctx, byte_buffer)
            decoded_lsb <<= 1
            decoded_lsb |= b
        decoded_data[i] = current_sign * (decoded_lsb + 1)
        
assert_array_equal(data, decoded_data)

### Solution
We report a solution which improves the coding efficiency of our simple arithmetic encoder. The first thing we note is the context associated with the bit planes of a symbol whose value is not zero. This context is denoted in the Python code cell as `lsb_ctx`. From the probability mass function above we note that values with magnitude equal to 1 have higher chance to appear than values with magnitude 2 and so on. Given that we encode 3 bins, it is reasonable to expect that the whole data distribution will adapt to be more skewed towards these values, with their less significant bits being more likely to be 1. From the code we note that the context `lsb_ctx` is initialised to index 113, which corresponds to a particular state in the probability state machine of the QM coder whereby the probability of the LPS is nearly 0.5 and there is no evolution to other probability states. From what we discussed, it makes sense to let the context `lsb_ctx` to evolve and adapt to the data distribution. By setting the initial state of this context to zero, we'll allow such an adaptivity. The following cell shows the fix to this design flaw.

In [ ]:
qme = QMEncoder()
qme.init_enc()

significance_ctx = Context(0, 0)
sign_ctx = Context(113, 0)
lsb_ctx = Context(0, 0)
for d in data:
    if not d:
        qme.code0(significance_ctx)
    else:
        qme.code1(significance_ctx)
        # Code the sign
        qme.code0(sign_ctx) if d > 0 else qme.code1(sign_ctx)
        d = abs(d) - 1
        bins = (bin(d)[2:]).zfill(3)
        for b in bins:
            qme.code0(lsb_ctx) if b == '0' else qme.code1(lsb_ctx)

qme.flush()
qme.insert_marker(endOfArithmeticCodingMarker)

byte_buffer = qme.get_byte_buffer()

original_bits = total_data * original_bps
coding_rate = len(byte_buffer) * 8
bps = coding_rate / total_data
print(f"Compression ratio: {original_bits / coding_rate:.2f}, bits per symbol: {bps}")

We notice that the bits per symbol went down to 3.14.

As an additional improvement, we can think to binarise the absolute value of non zero coefficients using unary coding and limiting unary coding to magnitude values less than or equal to three. The new value where unary coding is applied is the original absolute value of the data minus one. This would lead to strings such as: `0 = 0`, `1 = 10` and `2 = 110`. Each 1 in the string represents a positional bin and given that there are more data equal to one than those equal to two, it is reasonable to assume that the first bin will have a probability of 1 greater than that of the second bin. We have limited our unary coding to absolute values less than or equal to three so that a maximum of three is required (trade-off between coding efficiency and memory). The following Python cell code shows the extension of our arithmetic encoder. This time we managed to get the coding rate down to 2.97 bits per symbol.

In [ ]:
def unary(symbol):
    if symbol == 0:
        return '0'
    else:
        symbol = min(2, symbol)
        return '1' * symbol + '0'

qme = QMEncoder()
qme.init_enc()
    
significance_ctx = Context(0, 0)
sign_ctx = Context(113, 0)
lsb_ctx = Context(0, 0)
unary_ctx = [Context(0, 0), Context(0, 0), Context(0,0)]
for d in data:
    if not d:
        qme.code0(significance_ctx)
    else:
        qme.code1(significance_ctx)
        # Code the sign
        qme.code0(sign_ctx) if d > 0 else qme.code1(sign_ctx)

        d = abs(d) - 1
        bins_unary = unary(d)
        for i in range(len(bins_unary)):
            qme.code1(unary_ctx[i]) if bins_unary[i] == '1' else qme.code0(unary_ctx[i])
        if d > 2:
            bins = (bin(d - 2)[2:]).zfill(3)
            for b in bins:
                qme.code0(lsb_ctx) if b == '0' else qme.code1(lsb_ctx)

qme.flush()
qme.insert_marker(endOfArithmeticCodingMarker)

byte_buffer = qme.get_byte_buffer()
                
original_bits = total_data * original_bps
coding_rate = len(byte_buffer) * 8
bps = coding_rate / total_data
print(f"Compression ratio: {original_bits / coding_rate:.2f}, bits per symbol: {bps}")

As we may notice a simple change to the entropy coding workflow and addition of appropriate contexts allowed to bring the coding performance of our arithmetic coding engine closer to the Shannon's entropy.

## Conclusive remarks
The final part of our tutorial on arithmetic coding has been devoted to the use of a practical coding engine (i.e. the QM coder) to perform arithmetic coding of DCT quantised coefficients. Being the QM coder a binary coding engine, it requires to transform all non binary quantities so that we can feed strings of zeros and ones to the coding engine. Such a transformation is indeed the binarisation which can be performed either by using a naive base two coding or by exploiting the nature of the input data (e.g. whether are image pixels or transform coefficients). We reviewed a practical example of how quantised DCT coefficients are encoded in the JPEG compression standard when the QM coder is used. Aside from giving the opportunity to show how the statistics of these coefficients can be exploited to make the entropy coding process more efficient, it is hoped that the example also shown how the design of the entropy coding process in general and when arithmetic coding is used in particular, resembles an *art and craft* exercise where the expertise of the video codec engineer is key to the successful delivery of an efficient entropy coding module, optimising the different constraints associated with a given application scenario.